<a href="https://colab.research.google.com/github/yutawatabe/pyCGE/blob/main/EK_Exacthatalgebra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 反実仮想の計算

In [ ]:
# numpy, mathをインポート
import numpy as np # 行列計算のライブラリ
from math import gamma # 価格指数のため

#パラメーターを変更して解きなおすアプローチ

パラメーターをシミュレートして、Eaton Kortumモデルの均衡を計算します。均衡を計算するために賃金を更新する関数を再び導入し、最終的に均衡の賃金、貿易額を求めるところまで関数化します。

In [ ]:
def updatewage(w,theta,sigma,N,L_S,T,ｄ,psi):
  """
    パラメータと経済変数に基づいて賃金を更新します。

    この関数は、入力パラメータと経済変数に基づいて新しい賃金値を計算します。

  パラメータ:
      w (np.ndarray): 異なる国の現在の賃金率の配列。
      theta (float): 貿易の弾力性。
      sigma (float): 財の代替の弾力性。
      N (int): 国の数。
      L_S (np.ndarray): 各国の労働供給。
      T (np.ndarray): 各国の技術水準。
      d (np.ndarray): 国々間の貿易コスト。
      psi (float): 賃金調整パラメータ。

  戻り値:
      w_new (np.ndarray): 更新された国の賃金。
      Z (np.ndarray): 各国の超過労働需要
      P (np.ndarray): 価格指数。
      X (np.ndarray): 国々間の貿易フロー。
  """
  Xn = w * L

  ## 輸入シェアの計算
  pi = np.zeros((N,N)) # 輸入シェア（n国の総消費のうち、i国の財が何割を占めるか）。ここでは輸出国が最初の次元、輸出国が次の次元
  pi_num = np.zeros((N,N)) # 輸入シェアの分子部分
  Phi = np.zeros((N)) # 輸入シェアの分母部分
  for OR,DE in np.ndindex((N,N)):
    pi_num[OR,DE] = T[OR] * (w[OR] * ｄ[OR,DE]) ** (-theta)
    Phi[DE] += pi_num[OR,DE] # Phi[DE] = pi_den[DE] + pi_num[OR,DE]
  for OR,DE in np.ndindex((N,N)):
    pi[OR,DE] = pi_num[OR,DE] / Phi[DE]

  # 価格指数の計算
  P = gamma((theta + sigma - 1) / theta) **(1/(1-sigma)) * Phi ** (-1/theta)

  # 輸入額の計算
  X = np.zeros((N,N))
  for OR,DE in np.ndindex((N,N)):
    X[OR,DE] = pi[OR,DE] * Xn[DE]

  ## 労働超過需要の計算
  L_S = L
  L_D = np.zeros((N))
  for OR,DE in np.ndindex((N,N)):
    L_D[OR] += X[OR,DE] / w[OR] # L_D[OR] = L_D[OR] + X[OR,DE] / w[OR]
  Z = L_D - L_S

  # 賃金更新と基準化
  w_new = w * (1 + psi * (Z / L) )
  wgdp = np.sum(w_new * L)
  w_new = w_new / wgdp

  return w_new,Z,X,P

def solve_eqm(theta,sigma,N,L,T,d,psi,tol,maxiter):

  # 賃金と労働超過需要の初期化
  Z = np.ones((N))
  w = np.ones((N))

  iter = 1
  # 労働超過需要が十分に小さければLoopから抜け出す
  while max(np.abs(Z)) > tol and iter < maxiter:
    iter += 1
    w_old = np.copy(w)
    w,Z,_,_ = updatewage(w,theta=4,sigma=3,N=3,L_S=L,T=T,d=d,psi=0.1)
    if iter % 10 == 0:
      print(iter)
      #print(Z)
      #print(w)

  if iter == maxiter:
    print("Not done")
    return w,Z,P,X
  else:
    w,Z,X,P = updatewage(w,theta=4,sigma=3,N=3,L_S=L,T=T,d=d,psi=0.1)
    return w,Z,P,X


## End of Notebook